# Exploración de datos de "Inscripciones virtuales a los talleres de la Municipalidad Distrital de Jesús María"

Registro de las inscripciones virtuales a los talleres en la Municipalidad Distrital de Jesús María.

Los talleres de las Municipalidad Distrital de Jesús María son para los residentes del distrito tiene un descuento especial, y el público general también puede inscribirse a los talleres. Los horarios están habilitados durante todo el año según cronograma.

Los talleres que brinda la Municipalidad Distrital de Jesús María son Natación de 3 a 5 años, Natación de 6 años a más, futbol, taekondo, vóley, basket, karate, teatro, arte, guitarra, yoga, ballet, kpop, danza folclórica y muchos más. 

Este dataset esta caracterizado por:

Datos de la inscripción: Nombre del taller, local, días, horario y periodo
Datos del alumno: Nombre del alumno, fecha de nacimiento, edad y sexo
Precio para residentes de Jesús maría, precio para publico en general y precio total
Departamento, provincia, distrito y ubigeo donde se realizo los talleres

Fuente: https://www.datosabiertos.gob.pe/dataset/inscripciones-virtuales-los-talleres-de-la-municipalidad-distrital-de-jesús-maría-–-mjm

## 1. Preparativos para estudiar la inscripción a talleres de jesus maria

In [10]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

### 1.1. Instanciamos la sesión de spark

In [11]:
spark = SparkSession.builder \
    .appName("Analisis de Inscripciones a talleres de Jesús María") \
    .config("spark.jars", "/usr/local/spark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

### 1.2. Implementamos la estructura del CSV adjunto

In [12]:
schema = StructType([
    StructField("fecha_corte", StringType(), True),
    StructField("codigo_alumno", StringType(), True),
    StructField("fecha_nacimiento", StringType(), True),
    StructField("edad", IntegerType(), True),
    StructField("sexo", StringType(), True),
    StructField("taller", StringType(), True),
    StructField("local", StringType(), True),
    StructField("dias", StringType(), True),
    StructField("horario", StringType(), True),
    StructField("periodo", StringType(), True),
    StructField("precio_jesus_maria", DoubleType(), True),
    StructField("precio_publico_general", DoubleType(), True),
    StructField("precio_total", DoubleType(), True),
    StructField("departamento", StringType(), True),
    StructField("provincia", StringType(), True),
    StructField("distrito", StringType(), True),
    StructField("ubigeo", StringType(), True)
])

### 1.3 Carga de los datos del CSV

In [13]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("encoding", "UTF-8") \
    .schema(schema) \
    .load("/home/jlorenzor/data/inscripciones_talleres_jesus_maria.csv")

print("Vistazo al conjunto de datos:")
df.printSchema()
print(f"Total de registros: {df.count()}")

Vistazo al conjunto de datos:
root
 |-- fecha_corte: string (nullable = true)
 |-- codigo_alumno: string (nullable = true)
 |-- fecha_nacimiento: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- taller: string (nullable = true)
 |-- local: string (nullable = true)
 |-- dias: string (nullable = true)
 |-- horario: string (nullable = true)
 |-- periodo: string (nullable = true)
 |-- precio_jesus_maria: double (nullable = true)
 |-- precio_publico_general: double (nullable = true)
 |-- precio_total: double (nullable = true)
 |-- departamento: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- distrito: string (nullable = true)
 |-- ubigeo: string (nullable = true)

Total de registros: 1570


## 2. Transformación de datos

### 2.1. Cantidad de matriculados

In [14]:
df.groupBy("taller").count().orderBy(desc("count")).show(10, truncate=False)

+---------------------------+-----+
|taller                     |count|
+---------------------------+-----+
|NATACI�N A PARTIR DE 6 A�OS|413  |
|NATACI�N PATERA 3 A 5 A�OS |156  |
|FUTBOL MIXTO               |120  |
|TAEKWONDO                  |113  |
|B�SQUET                    |108  |
|VOLEY                      |108  |
|VOLEY M�STER               |68   |
|AJEDREZ                    |55   |
|CLOWN                      |50   |
|KARATE                     |37   |
+---------------------------+-----+
only showing top 10 rows



### 2.2. Distribución de edad por sexo

In [15]:
df.groupBy("sexo").agg(
    avg("edad").alias("avg_age"),
    min("edad").alias("min_age"),
    max("edad").alias("max_age"),
    count("*").alias("total")
).show()

+---------+------------------+-------+-------+-----+
|     sexo|           avg_age|min_age|max_age|total|
+---------+------------------+-------+-------+-----+
|MASCULINO| 9.156626506024097|      1|     50|  747|
| FEMENINO|11.952612393681653|    -11|     71|  823|
+---------+------------------+-------+-------+-----+



### 2.3. Tipos de precios

In [16]:
df.select("precio_total", "precio_jesus_maria", "precio_publico_general").describe().show()

+-------+------------------+------------------+----------------------+
|summary|      precio_total|precio_jesus_maria|precio_publico_general|
+-------+------------------+------------------+----------------------+
|  count|              1570|              1570|                  1570|
|   mean| 125.2547770700637|121.68789808917198|      3.56687898089172|
| stddev|35.757736723818645|38.028427443930816|     26.47828791391427|
|    min|             100.0|               0.0|                   0.0|
|    max|             250.0|             250.0|                 200.0|
+-------+------------------+------------------+----------------------+



### 2.4. Distribución geografica

In [17]:
df.groupBy("distrito").count().orderBy(desc("count")).show()

+-----------+-----+
|   distrito|count|
+-----------+-----+
|JESUS MARIA| 1570|
+-----------+-----+



Detenemos la sesión de spark

In [18]:
spark.stop()